In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import pandas as pd
import spacy


In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
# Cargar los datos de entrenamiento
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[0, 1, 2, 3], engine='python')

# Cargar el modelo de spaCy para tokenizar oraciones
nlp = spacy.load("es_core_news_sm")

## **ZERO SHOT LEARNING CLASSIFICATION 4 LABELS**
4 CLASSES


*   'related to an outdoor space with positive effect'
*   'related to an outdoor space with neutral effect'
*   'related to an outdoor space with negative effect'
*   'not related to an outdoor space'

In [ ]:
# Función para obtener los enunciados con las keywords
def extract_keyword_sentences(row):
    keywords = set(row['keyword'].split(', '))  # Convertir las keywords en un conjunto para eliminar duplicados
    text = row['text']
    doc = nlp(text)
    found_sentences = set()  # Conjunto para almacenar las oraciones encontradas
    for sentence in doc.sents:
        # Verificar si la oración contiene al menos una de las keywords
        if any(keyword.lower() in sentence.text.lower() for keyword in keywords):
            found_sentences.add(sentence.text.strip())  # Agregar la oración al conjunto de oraciones encontradas
    return '. '.join(found_sentences) if found_sentences else None  # Unir las oraciones encontradas en un solo texto

In [ ]:
# Filtrar y actualizar el texto con los enunciados que contienen las keywords
data['text'] = data.apply(lambda row: extract_keyword_sentences(row), axis=1)
datatext = data['text'].astype(str)
print(len(datatext))
print(datatext[0:10])

1800
0    Afterwards, I want to make a run at young love...
1    Since I was still semi-anxious, I didnt really...
2    Almost everything costs money (who woulda thou...
3    I got to class early I walked in filled with c...
4    So anyway I stepped outside to speak to them a...
5    Imagine X guy running up to you suddenly: "Hey...
6    That made everything worse bc she was my only ...
7    Sitting at the house while my crush is on a ro...
8    When I imagine myself as an older lady, I pict...
9    A yes or no question and yet here i am a week ...
Name: text, dtype: object


In [ ]:
# Convertir cada texto en el formato deseado
formatted_texts = datatext.apply(lambda text: f"('{text}')")
print(len(formatted_texts))

1800



---

## **MODEL: sileod/deberta-v3-base-tasksource-nli**

https://huggingface.co/sileod/deberta-v3-base-tasksource-nli?candidate_labels=positive%2C+negative%2C+neutral%2C+unrelated&multi_class=false&text=Flirting+%3F%3F.+hi+people+of+SA+%21%21%21+I%27m+pretty+new+here%2C+and+I+need+some+advice...+Okay%2C+I+am+a+17F+and+I+like+to+go+on+walks.+I+started+like+a+month+ago%2C+and+I+really+enjoy+having+that+time+to+myself+to+just+think+and+breath+%28I+live+by+the+ocean%2C+so+beach+walks%21%29+Anyways%2C+pretty+much+every+day+I+go+on+a+walk+I+get+a+guy+flirting+with+me+or+whistling%2C+and+I+don%27t+really+know+how+to+deal+with+it.+Like%2C+I%27m+not+wearing+anything+provocative%2C+I+just+want+to+walk+in+the+sand+and+look+at+cute+dogs.+Idk+what+to+do%2C+please+does+someone+have+any+advice%3F+Sidenote+%3A+if+it%27s+a+guy+I+might+be+into+and+he%27s+flirting%2C+what+should+I+do%3F+I%27m+so+awkward+I+never+know+what+to+say+Thanks+%21%21%21

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",model="sileod/deberta-v3-base-tasksource-nli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_5 = []
for text in formatted_texts:
    output_5 = classifier(text, candidate_labels=['related to a natural outdoor space with positive effect', 'related to a natural outdoor space with negative effect','related to a natural outdoor space with neutral effect', 'not related to a natural outdoor space'])
    predictions_5.append(output_5)
    print(output_5)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': "('Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out.')", 'labels': ['not related to a natural outdoor space', 'related to a natural outdoor space with negative effect', 'related to a natural outdoor space with positive effect', 'related to a natural outdoor space with neutral effect'], 'scores': [0.5177621245384216, 0.171988382935524, 0.16759775578975677, 0.14265176653862]}
{'sequence': "('Since I was still semi-anxious, I didnt really talk anything besides saying hi, sup, and telling her that its boring being home, that I can be shy, etc, she suggested that I could hang around outside but I didnt know if she could hang with')", 'labels': ['related to a natural outdoor space with positive effect', 'related to a natural outdoor space with neutral effect', 'not related to a natural outdoor space', 'related to a natural o

In [ ]:
# Contar el total de clasificaciones como 'outdoor space' y 'not outdoor space'
total_outdoor_space_pos = sum(1 for pred in predictions_5 if pred['labels'][0] == 'related to a natural outdoor space with positive effect')
total_outdoor_space_neg = sum(1 for pred in predictions_5 if pred['labels'][0] == 'related to a natural outdoor space with negative effect')
total_outdoor_space_neu = sum(1 for pred in predictions_5 if pred['labels'][0] == 'related to a natural outdoor space with neutral effect')
total_not_outdoor_space = sum(1 for pred in predictions_5 if pred['labels'][0] == 'not related to a natural outdoor space')

# Imprimir los resultados
print("Total de clasificados como 'outdoor space with positive effect':", total_outdoor_space_pos)
print("Total de clasificados como 'outdoor space with negative effect':", total_outdoor_space_neg)
print("Total de clasificados como 'outdoor space with neutral effect':", total_outdoor_space_neu)
print("Total de clasificados como 'not outdoor space':", total_not_outdoor_space)

Total de clasificados como 'outdoor space with positive effect': 519
Total de clasificados como 'outdoor space with negative effect': 301
Total de clasificados como 'outdoor space with neutral effect': 76
Total de clasificados como 'not outdoor space': 904


In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas reales del conjunto de datos
y_true = data['label']

# Convertir las etiquetas predichas a números según el formato especificado en tu pregunta
label_map = {
    'related to a natural outdoor space with positive effect': 1,
    'related to a natural outdoor space with neutral effect': 2,
    'related to a natural outdoor space with negative effect': 3,
    'not related to a natural outdoor space': 0
}
# Mapear las etiquetas predichas al formato numérico
y_pred = [label_map[prediction['labels'][0]] for prediction in predictions_5]

# Obtener el classification report
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.81      0.65      0.72      1131
           1       0.23      0.76      0.36       160
           2       0.25      0.05      0.08       395
           3       0.21      0.55      0.30       114

    accuracy                           0.52      1800
   macro avg       0.38      0.50      0.37      1800
weighted avg       0.60      0.52      0.52      1800




---

## **MODEL: MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli**


In [ ]:
# Cargar los datos de entrenamiento
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[0, 1, 2, 3], engine='python')
datatext = data['text'].astype(str)
print(len(datatext))
formatted_texts = datatext.apply(lambda text: f"('{text}')")
print(len(formatted_texts))

1800
1800


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_6 = []
for text in formatted_texts:
    output_6 = classifier(text, truncation=True, max_length = 512, candidate_labels=['related to an outdoor space with positive effect',
                                                  'related to an outdoor space with neutral effect',
                                                  'related to an outdoor space with negative effect',
                                                  'not related to an outdoor space'])
    predictions_6.append(output_6)
    print(output_6)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': "('21/m. I want to experience young love, but I've never had a relationship before. Am I going to miss out?. I have bad social anxiety and because of it not much social experience, but I think I can work through that within a year. Afterwards, I want to make a run at young love, but since I've never had any experience, I feel like I'll be building a bridge that will take a few years to build, and by that point I would have missed out. Given my situation, can I experience young love? Am I just going to have to miss out given my situation? tl;dr -- want to experience young love, but fear I don't have the experience to make it happen in time. Is this an unrealistic goal?')", 'labels': ['not related to an outdoor space', 'related to an outdoor space with neutral effect', 'related to an outdoor space with negative effect', 'related to an outdoor space with positive effect'], 'scores': [0.9966346025466919, 0.001441286876797676, 0.0010622780537232757, 0.0008618143619969487]}
{'se

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas reales del conjunto de datos
y_true = data['label']

# Convertir las etiquetas predichas a números según el formato especificado en tu pregunta
label_map = {
    'not related to an outdoor space': 0,
    'related to an outdoor space with positive effect': 1,
    'related to an outdoor space with neutral effect': 2,
    'related to an outdoor space with negative effect': 3
}

# Mapear las etiquetas predichas al formato numérico
y_pred = [label_map[prediction['labels'][0]] for prediction in predictions_6]

# Obtener el classification report
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.77      0.77      0.77      1131
           1       0.36      0.54      0.44       160
           2       0.28      0.05      0.08       395
           3       0.16      0.52      0.25       114

    accuracy                           0.58      1800
   macro avg       0.40      0.47      0.39      1800
weighted avg       0.59      0.58      0.56      1800



In [ ]:
print(y_pred)

[0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 3, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2, 3, 0, 0, 3, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 3, 3, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 3, 0, 2, 3, 0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 